In [1]:
from automatic_variable_mapping import corpus, vocab_similarity
import pandas as pd
from functools import partial
import time
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/harrison/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/harrison/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from automatic_variable_mapping.vocab_similarity import default_pairable, partition
num_cpus = 14

In [3]:
reload(vocab_similarity)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/harrison/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/harrison/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<module 'automatic_variable_mapping.vocab_similarity' from 'automatic_variable_mapping/vocab_similarity.pyc'>

In [4]:
def matching_groups(data, group_col, corpora, pair_id, ref_id):
    corpus_doc_ids = [doc_id for doc_id, _ in corpora]
    ref_idx = corpus_doc_ids.index(ref_id)
    pair_idx = corpus_doc_ids.index(pair_id)
    return data[pair_idx][group_col] == data[ref_idx][group_col]


def pairable_by_group(data, group_col, corpus_doc_ids, score, pair_id, _, ref_id):
    return vocab_similarity.default_pairable(score, pair_id, None, ref_id) and not matching_groups(data, group_col, corpus_doc_ids, pair_id, ref_id)


def calc_score_results(data_file, doc_cols, doc_col_key, ref_id_col, filter_file, mult_corpora=False, corpora_col=None):
    print data_file
    data = pd.read_csv(data_file, sep=",",quotechar='"', na_values="",low_memory=False)
    if mult_corpora:
        corpora_data = partition(data, corpora_col)
    else:
        corpora_data = [data]
    
    if filter_file != data_file:
        filter_data = pd.read_csv(filter_file,
                                  sep=",",
                                  quotechar='"',
                                  na_values="",
                                  low_memory=False)
    else:
        filter_data = data

    #idc = list(doc_cols)
    #idc.append(ref_id_col)
    #print corpora_data[0][idc]
    corpora = corpus.build_corpora(doc_cols, corpora_data, ref_id_col, num_cpus=num_cpus)
    tfidf_matrix = corpus.calc_tfidf(corpora)

    scores = vocab_similarity.VariableSimilarityCalculator(filter_data[ref_id_col],
                                                           pairable=default_pairable)

    scores.init_cache()
    if mult_corpora:
        result = scores.score_variables(corpora, tfidf_matrix, num_cpus=num_cpus)
        tfidf_label = "multtfcdf"
    else:
        result = scores.score_variables(corpora[0], tfidf_matrix, num_cpus=num_cpus)
        tfidf_label = "tfidf"
    result = result[result["score"] != 0]
    
    result.to_csv("/run/media/harrison/external-hd/data/automatic-variable-mapping-results/var_txt_sim_" + tfidf_label + "_scores_obs_heart_studies_" + doc_col_key + ".csv")
    return None

In [5]:
obs_data_file = "~/Dropbox/tiff_laura_shared/var_doc_obs_heart_studies_dbGaP_NLP.csv"
obs_man_file = "~/Dropbox/tiff_laura_shared/manual_concept_var_mappings_dbGaP_obs_heart_studies_NLP.csv"
ref_id_col = 'dbGaP_studyID_datasetID_varID'
doc_cols_inputs = {'desc': ['variable_description'],
                   'units': ['units'],
                   'coding': ['variable_coding_counts_distribution'],
                   'desc_units': ['variable_description', 'units'],
                   'desc_coding': ['variable_description', 'units', 'variable_coding_counts_distribution'],
                   'desc_units_coding': ['variable_description', 'units', 'variable_coding_counts_distribution']
                  }


In [6]:
dt = pd.read_csv(obs_man_file,
                       sep=",",
                       quotechar='"',
                       na_values="",
                       low_memory=False)
dt
dt.columns

Index([u'study', u'cohort_dbGaP', u'dbGaP_studyID_datasetID',
       u'dbGaP_dataset_label', u'concept', u'data_desc', u'varID', u'var_desc',
       u'units', u'var_coding_counts_distribution', u'var_doc_id',
       u'concept_id', u'dbGaP_studyID_datasetID_varID', u'var_coding_labels'],
      dtype='object')

In [7]:
def calc_scores_doc_cols(data_file, doc_cols_inputs, ref_id_col, filter_file, mult_corpora=False, corpora_col=None):
    scores_dfs = list()
    for key in doc_cols_inputs:
        score_name = "score_" + key
        data = pd.read_csv(data_file,
                       sep=",",
                       quotechar='"',
                       na_values="",
                       low_memory=False)
        scores_df = calc_score_results(data_file, doc_cols_inputs[key], key, ref_id_col, filter_file, mult_corpora, corpora_col)
        scores_df = scores_df.rename({'score': score_name}, axis=1)
        scores_dfs.append(scores_df)

    #scores_merged = reduce(lambda left, right: pd.merge(left, right, on=[ref_id_col],
    #                                               how='outer'), scores_dfs)
    #return(scores_merged)
    return(scores_dfs)

In [ ]:
obs_scores_tfidf = calc_scores_doc_cols(obs_data_file, doc_cols_inputs, ref_id_col, obs_man_file)

~/Dropbox/tiff_laura_shared/var_doc_obs_heart_studies_dbGaP_NLP.csv


automatic_variable_mapping/corpus.py:83: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  corpus = list (tqdm.tqdm(pool.imap(partial(helper, doc_col), corpus_data[cols].as_matrix()), total=corpus_data.shape[0]))
100%|██████████| 105611/105611 [00:34<00:00, 3066.30it/s]


Finding valid pair ids
Pair ids: 105611
Finding valid ref ids
Ref ids: 1709
Multiplying matrices
LHS: (105611, 60261)
RHS: (1709, 60261)
Sim Matrix: (1709, 105611)


In [ ]:
obs_scores_tfmcdf = calc_scores_doc_cols(obs_data_file, {"desc": ['variable_description']}, ref_id_col, obs_man_file, mult_corpora=True, corpora_col='study')

In [25]:
#TO DO add standard and add same above but for clinical trials

#doc_col = list("var_desc_1”, “units_1", “var_coding_counts_distribution_1")
score_file = 'tests/test_var_similarity_scores_rank_data.csv'

In [60]:
result = obs_scores_tfidf[0]
result

,reference var,paired var,score_desc
0,phs000007.v26.pht000009.v2.MF5,phs000287.v5.pht001452.v1.GRADE01,1.000000
1,phs000007.v26.pht000009.v2.MF5,phs000287.v5.pht001450.v1.GRADE01,1.000000
2,phs000007.v26.pht000009.v2.MF5,phs000287.v5.pht001490.v1.GRADE01,1.000000
3,phs000007.v26.pht000009.v2.MF5,phs000007.v26.pht000031.v7.B43,0.870514
4,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001118.v8.momschl2,0.657877
5,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001119.v8.momschl3,0.657877
6,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001118.v8.dadschl2,0.653778
7,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001119.v8.dadschl3,0.653778
8,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht001116.v10.educ1,0.613257
9,phs000007.v26.pht000009.v2.MF5,phs000209.v13.pht003087.v1.educ1,0.613257


In [61]:
result = result[result["score_desc"] != 0]
result = result.rename(columns={"score_desc":"score"})

In [62]:
score_file = 'tests/test_var_similarity_scores_rank_data.csv'
score_data = pd.read_csv(score_file)
score_data = score_data[score_data["concept"] == "Age (years)"]
score_data

,concept,dbGaP_studyID_datasetID_varID_1,dbGaP_studyID_datasetID_1,dbGaP_studyID_datasetID_varID_2,dbGaP_studyID_datasetID_2,correctMatchTrue,score_desc,rank_score_desc,score_codeLab_relativeDist,rank_score_codeLab_relativeDist,Unnamed: 10
53775,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.age1c,phs000209.v13.pht001111.v4,True,0.785504,1.0,0.600015,16.0,NaN
53776,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.agequit1,phs000209.v13.pht001111.v4,False,0.324743,14.0,0.782636,2.0,NaN
53777,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.cholage1,phs000209.v13.pht001111.v4,False,0.307649,21.0,0.788611,1.0,NaN
53778,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.hystage1,phs000209.v13.pht001111.v4,False,0.456302,3.0,0.608377,14.0,NaN
53779,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.aspsage1,phs000209.v13.pht001111.v4,False,0.303891,22.0,0.739248,3.0,NaN
53780,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.pipageq1,phs000209.v13.pht001111.v4,False,0.311395,17.0,0.689516,4.0,NaN
53781,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.dbage1,phs000209.v13.pht001111.v4,False,0.308856,20.0,0.680037,7.0,NaN
53782,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.bphxage1,phs000209.v13.pht001111.v4,False,0.298671,25.0,0.688714,5.0,NaN
53783,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.aspeage1,phs000209.v13.pht001111.v4,False,0.296889,26.0,0.686862,6.0,NaN
53784,Age (years),phs000007.v26.pht003099.v4.age1,phs000007.v26.pht003099.v4,phs000209.v13.pht001111.v4.cgrageq1,phs000209.v13.pht001111.v4,False,0.311582,16.0,0.657114,8.0,NaN


In [64]:
score_comb = pd.merge(score_data, result, how='left', left_on=[ 'dbGaP_studyID_datasetID_varID_1', 'dbGaP_studyID_datasetID_varID_2' ], right_on=[ 'reference var', 'paired var' ]).round(6)
score_comb
dt = score_comb[["score_desc", "score", 'dbGaP_studyID_datasetID_varID_1', 'dbGaP_studyID_datasetID_varID_2', "reference var", "paired var"]]
dt = dt[~np.isnan(dt["score"])]
#dt = dt[dt['score'] != dt['score_desc']]
dt.sort_values("score")

,score_desc,score,dbGaP_studyID_datasetID_varID_1,dbGaP_studyID_datasetID_varID_2,reference var,paired var
56192,0.020119,0.020119,phs000287.v5.pht001466.v1.AGE2,phs000209.v13.pht001116.v10.total_cis_e1c,phs000287.v5.pht001466.v1.AGE2,phs000209.v13.pht001116.v10.total_cis_e1c
53533,0.024988,0.024988,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht001111.v4.cepgfr1c,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht001111.v4.cepgfr1c
53949,0.025264,0.025264,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht003091.v3.cepgfr5c,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht003091.v3.cepgfr5c
53834,0.025264,0.025264,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht003086.v3.cepgfr5c,phs000280.v3.pht004065.v1.V4AGE41,phs000209.v13.pht003086.v3.cepgfr5c
48084,0.025491,0.025491,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht001111.v4.cepgfr1c,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht001111.v4.cepgfr1c
51986,0.025580,0.025580,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht001111.v4.cepgfr1c,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht001111.v4.cepgfr1c
48452,0.025773,0.025773,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht003086.v3.cepgfr5c,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht003086.v3.cepgfr5c
48580,0.025773,0.025773,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht003091.v3.cepgfr5c,phs000280.v3.pht004062.v1.V2AGE22,phs000209.v13.pht003091.v3.cepgfr5c
52431,0.025863,0.025863,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht003091.v3.cepgfr5c,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht003091.v3.cepgfr5c
52310,0.025863,0.025863,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht003086.v3.cepgfr5c,phs000280.v3.pht004064.v1.V3AGE31,phs000209.v13.pht003086.v3.cepgfr5c
